# E-commerce - Выявление профилей потребления

Мы работаем в интернет-магазине товаров для дома «Пока все ещё тут». Поступил запрос от менеджера магазина, который предложил улучшить потребительский опыт для более успешной работы магазина.

**Цель исследования:** 
- Увеличить выручку магазина.

**Задача:**
- Сегментировать покупателей по профилю потребления для выявления целевых групп, для которых можно будет подготовить персонализированные предложения.

**Ход исследования:**
- Получим необходимые данные из файла /datasets/ecom_dataset_upd.csv.
- Выполним предварительную обработку данных, добавим необходимые столбцы.
- Проанализируем основные показатели магазина их их динамику, построим визуализации.
- Сформируем группы покупателей на основе покупок.
- Подготовим презентацию для менеджера магазина c рекомендациями по работе с разными группами покупателей.

## Предобработка данных
### Общая информация

In [56]:
# Вводим библиотеки
import pandas as pd
from scipy import stats as st
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math as mth
from plotly import graph_objects as go
import plotly.express as px
import plotly

In [57]:
#pd.set_option('display.max_columns', None)  
pd.options.display.max_colwidth = 150 
#pd.set_option('display.max_rows', 200)
#pd.options.display.max_rows = 20

In [58]:
# Открываем файл
try:
    data = pd.read_csv('/datasets/ecom_dataset_upd.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/ecom_dataset_upd.csv')

display(data.head())
data.info()

,date,customer_id,order_id,product,quantity,price
0,2018100100,ee47d746-6d2f-4d3c-9622-c31412542920,68477,"Комнатное растение в горшке Алое Вера, d12, h30",1,142.0
1,2018100100,ee47d746-6d2f-4d3c-9622-c31412542920,68477,"Комнатное растение в горшке Кофе Арабика, d12, h25",1,194.0
2,2018100100,ee47d746-6d2f-4d3c-9622-c31412542920,68477,Радермахера d-12 см h-20 см,1,112.0
3,2018100100,ee47d746-6d2f-4d3c-9622-c31412542920,68477,Хризолидокарпус Лутесценс d-9 см,1,179.0
4,2018100100,ee47d746-6d2f-4d3c-9622-c31412542920,68477,Циперус Зумула d-12 см h-25 см,1,112.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7474 entries, 0 to 7473
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   date         7474 non-null   int64  
 1   customer_id  7474 non-null   object 
 2   order_id     7474 non-null   int64  
 3   product      7474 non-null   object 
 4   quantity     7474 non-null   int64  
 5   price        7474 non-null   float64
dtypes: float64(1), int64(3), object(2)
memory usage: 350.5+ KB


У нас есть таблица **data** с 6ю столбцами, которая описывает транзакции интернет-магазина. Можно заметить, что количество значений в столбцах одинаково, значит в данных нет пропущенных значений. Названия столбцов соответствуют хорошему стилю. Также видим, что не все столбцы имеют подходящий тип данных (столбец с датой).

Согласно документации к данным:
* `date` — дата заказа;           
* `customer_id` — идентификатор покупателя;  
* `order_id` — идентификатор заказа;
* `product` — наименование товара;
* `quantity` — количество товара в заказе;
* `price` — цена товара.

Предварительно можно говорить, что данных достаточно для анализа продаж в этом магазине. Но предварительно необходимо подготовить и обработать их.

### Корректировка типов данных, добавление новых столбцов

Столбец `date` задержит информацию о дате, а также о часе транзакции. Преобразуем тип этого столбца в datetime64, а также добавим отдельный столбец, который будет содержать только месяц транзакции `month`.

Также добавим столбец с общей стоимостью заказа, умножив цену товара на количество.

In [59]:
data['date'] = pd.to_datetime(data['date'], format="%Y%m%d%H")
data['month'] = data['date'].astype('datetime64[M]') 
data['price_total'] = data['quantity'] * data['price']

In [60]:
data.head()

,date,customer_id,order_id,product,quantity,price,month,price_total
0,2018-10-01,ee47d746-6d2f-4d3c-9622-c31412542920,68477,"Комнатное растение в горшке Алое Вера, d12, h30",1,142.0,2018-10-01,142.0
1,2018-10-01,ee47d746-6d2f-4d3c-9622-c31412542920,68477,"Комнатное растение в горшке Кофе Арабика, d12, h25",1,194.0,2018-10-01,194.0
2,2018-10-01,ee47d746-6d2f-4d3c-9622-c31412542920,68477,Радермахера d-12 см h-20 см,1,112.0,2018-10-01,112.0
3,2018-10-01,ee47d746-6d2f-4d3c-9622-c31412542920,68477,Хризолидокарпус Лутесценс d-9 см,1,179.0,2018-10-01,179.0
4,2018-10-01,ee47d746-6d2f-4d3c-9622-c31412542920,68477,Циперус Зумула d-12 см h-25 см,1,112.0,2018-10-01,112.0


После коррекции типа столбца с датой, проверим, за какой период у нас имеются данные.

In [61]:
print('Минимальная дата транзакции:', data['date'].min())
print('Максимальная дата транзакции:', data['date'].max())

Минимальная дата транзакции: 2018-10-01 00:00:00
Максимальная дата транзакции: 2020-01-31 15:00:00


У нас в наличии данные за период с 1 октября 2018 по 31 января 2020 года. 

### Удаление тестового заказа

Нам известно, что самый большой заказ (на самую большую сумму) - тестовый. Удалим его.

In [62]:
data.groupby('order_id').agg({'price_total': 'sum'})\
              .reset_index().sort_values(by = 'price_total', ascending=False)

,order_id,price_total
2045,71743,675000.0
221,14833,114750.0
1658,70960,65220.0
502,68760,50770.0
933,69531,49668.0
...,...,...
3192,107790,10.0
3057,105745,10.0
3105,106536,10.0
3437,111690,10.0


In [63]:
data = data[data['order_id'] != 71743]

### Проверка на наличие дубликатов

В таблице огромное количество наименований продуктов. Приведем столбец product к нижнему регистру на случай, если вдруг одни и те же наименования записаны в разных регистрах, а также уберем букву "ё".

In [64]:
data['product']=data['product'].str.lower()
data['product']=data['product'].str.replace('ё', 'е')
print('Количество уникальных наименований продуктов:', data['product'].nunique())

Количество уникальных наименований продуктов: 2340


In [65]:
# Проверяем наличие явных дубликатов в датафреймах
print ('Количество полных дубликатов:', data.duplicated().sum())

Количество полных дубликатов: 0


Полных дубликатов не оказалось. Но мы должны также проверить, что если строка с товаром (количество + цена) повторяется в одном заказе с разными датами или с разными покупателями, то это тоже дубликат. Будем от них избавляться.

In [66]:
print ('Количество неполных дубликатов:', data.duplicated(subset=['order_id', 'product', 'quantity', 'price']).sum())

Количество неполных дубликатов: 1897


In [67]:
# Избавляемся от дубликатов и проверям, сколько осталось строк в таблице
data = data.drop_duplicates(subset=['order_id', 'product', 'quantity', 'price'])
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5576 entries, 0 to 7473
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   date         5576 non-null   datetime64[ns]
 1   customer_id  5576 non-null   object        
 2   order_id     5576 non-null   int64         
 3   product      5576 non-null   object        
 4   quantity     5576 non-null   int64         
 5   price        5576 non-null   float64       
 6   month        5576 non-null   datetime64[ns]
 7   price_total  5576 non-null   float64       
dtypes: datetime64[ns](2), float64(2), int64(2), object(2)
memory usage: 392.1+ KB


Теперь посмотрим на те заказы, на которые приходятся более двух дат или более двух уникальных пользователей.

In [68]:
data_by_order_id = data.groupby('order_id').agg({'date': 'nunique', 'customer_id': 'nunique'}).reset_index()
data_by_order_id.columns = ['order_id', 'date_count', 'customer_id_count']
data_by_order_id[(data_by_order_id['date_count'] >1) | (data_by_order_id['customer_id_count'] >1)]

,order_id,date_count,customer_id_count
14,14500,2,1
108,14649,2,1
221,14833,2,1
267,14898,2,1
796,69282,2,1
902,69485,2,2
931,69527,2,1
1031,69750,2,1
1055,69807,2,1
1080,69861,2,1


In [69]:
data[data['order_id'] == 69485]

,date,customer_id,order_id,product,quantity,price,month,price_total
1282,2018-12-20 16:00:00,5be0f4b4-947f-4672-b147-73ef5936ecd4,69485,набор столовых приборов 24 предмета saint germain domenik dm9638,1,1087.0,2018-12-20 16:00:00,1087.0
1292,2018-12-21 11:00:00,266beb08-ec9b-43b4-89bb-ac60fa23ffee,69485,набор столовых приборов 24 предмета farandole domenik dm9635,1,1274.0,2018-12-21 11:00:00,1274.0


In [70]:
data[data['order_id'] == 70903]

,date,customer_id,order_id,product,quantity,price,month,price_total
2491,2019-03-28 10:00:00,5e5a3e1a-0388-4131-a770-f6e3b696985b,70903,средство против засоров в системах водопровода decs антизасор 1000 мл россия 4660015923303,1,104.0,2019-03-28 10:00:00,104.0
3344,2019-05-01 01:00:00,718316fc-bdf3-4d1d-b8f1-a5c763238ff4,70903,средство против засоров в системах водопровода decs антизасор 1000 мл россия 4660015923303,1,142.0,2019-05-01 01:00:00,142.0


Будем считать ошибками выгрузки те заказы, на которые приходится по 2 идентификатора покупателя и по 2 даты одновременно. Заказы, на которые приходится две даты (точнее две даты с учетом часа), оставим.

In [71]:
data = data[(data['order_id'] != 69485) & (data['order_id'] != 70903)]

### Анализ столбцов с числовыми значениями

Рассмотрим подробнее столбцы `quantity` и `price`.

In [72]:
data.boxplot('price')
data['price'].describe()

count     5572.00000
mean       530.21448
std        973.08164
min          9.00000
25%         90.00000
50%        150.00000
75%        524.00000
max      14917.00000
Name: price, dtype: float64

По диаграмме размаха наблюдаем, что медианное значение цены за товар составляет 150 р. Медианное значение сильно отличается от среднего, а значит в таблице есть значения, сильно отличающиеся от большинства. Верхний "ус" диаграммы упирается примерно в 1175 (=1,5 межквартального размаха + 3-й квартиль), что считается границей нормального размаха. Все, что выше этой цены, относится к необычно высоким ценам за товар. 

Таким образом, мы понимаем, что в основном стоимость товаров невысока, но также есть категория товаров, относящаяся к более высокому ценовому сегменту.

In [73]:
data.boxplot('quantity')
data['quantity'].describe()

count    5572.000000
mean        2.410445
std         9.588821
min         1.000000
25%         1.000000
50%         1.000000
75%         1.000000
max       334.000000
Name: quantity, dtype: float64

Видим, что в основном покупают по одной единице товара. Но также встречаются и оптовые закупки. Максимальное количество позиций товара - 334 единицы.

## Исследовательский анализ
Проанализируем основные показатели интернет-магазина и их динамику. 
### Количество заказов и выручка по покупателям
Рассмотрим подробнее данные по каждому покупателю.

In [74]:
# Смотрим, сколько заказов и какая выручка приходится на одного покупателя
data_by_customers = data.groupby('customer_id').agg({'order_id': 'nunique', 'price_total': 'sum'}).reset_index()
data_by_customers.columns = ['customer_id', 'order_count', 'revenue_by_customer']

data_by_customers.sort_values(by='order_count', ascending=False).head(10)

,customer_id,order_count,revenue_by_customer
1896,c971fb21-d54c-4134-938f-16b62ee86d3b,126,159508.0
732,4d93d3f6-8b24-403b-a74b-f5173e40d7db,35,57278.0
1087,73d1cd35-5e5f-4629-8cf2-3fda829d4e58,17,21361.0
1730,b7b865ab-0735-407f-8d0c-31f74d2806cc,7,3779.0
17,0184f535-b60a-4914-a982-231e3f615206,5,5891.0
1801,bea7a833-2074-42db-bc49-4457abd3c930,4,4409.0
2124,e0535076-6270-4df2-8621-cb06264a94fa,4,636.0
689,498f12a4-6a62-4725-8516-cf5dc9ab8a3a,4,41900.0
2284,f163e581-59ba-4022-99db-e0973c7497c0,3,2511.0
998,6b0c6cfb-7717-4c34-8535-bbc6e2b2c758,3,660.0


In [75]:
data_by_customers.describe()

,order_count,revenue_by_customer
count,2428.000000,2428.000000
mean,1.448929,1625.087074
std,2.690050,4328.826555
min,1.000000,15.000000
25%,1.000000,394.500000
50%,1.000000,838.500000
75%,2.000000,1798.250000
max,126.000000,159508.000000


Видим, что медианные значения количества заказов на покупателя и выручки на покупателя в 1,5-2 раза отличаются от средних значений. Это значит, что имеют место значения, сильно отличающиеся от большинства. Особенно выделяются три покупателя с самым большим количеством заказов (126, 35, 17).

В среднем, на одного покупателя приходится 1 заказ, а выручка составляет 838 на покупателя.


### Общее количество заказов, покупателей

Посмотрим, что у нас получилось, учитывая удаленные аномальные значения.

In [76]:
print('Всего наименований продуктов:', data['product'].nunique())
print('Всего заказов:', data['order_id'].nunique())
print('Всего уникальных покупателей:', data['customer_id'].nunique())

Всего наименований продуктов: 2337
Всего заказов: 3518
Всего уникальных покупателей: 2428


### Разделение покупателей по количеству заказов

Выше мы выяснили, что в основном на 1 пользователя приходится 1 заказ. Интересно рассмотреть подробнее группу пользователей, которые совершали 2 и более заказов. Ведь повторное возвращение в магазин - это важная показатель лояльности покупателя. Для этого добавим новый столбец `customer_type` с двумя вариантами: "2 и более заказов" и "1 заказ". Покупателей с 1 заказом будем называть одиночными. Покупателей с 2 и более заказами - повторными.

In [77]:
#Покупатели, совершившие 2 и более заказов
repeat_customers = data_by_customers['customer_id'][data_by_customers['order_count'] >1].reset_index()
repeat_customers['customer_type'] = '2 и более заказов'
repeat_customers

,index,customer_id,customer_type
0,6,005ba170-45e8-42de-93f7-192481ae2659,2 и более заказов
1,8,00c0f92a-121a-4883-a78d-2c7b5fe187de,2 и более заказов
2,10,00cd704a-a65c-4865-b39a-779f1b4f1d1a,2 и более заказов
3,12,00ff5327-0fba-481c-8a07-47a95093a213,2 и более заказов
4,17,0184f535-b60a-4914-a982-231e3f615206,2 и более заказов
...,...,...,...
879,2413,fe3e6382-3272-4045-8601-c9acfc08006f,2 и более заказов
880,2419,fef19457-5e7b-4d5d-b031-806041bc251a,2 и более заказов
881,2421,ff31d802-dae6-484d-a3dc-c9723f1cb538,2 и более заказов
882,2422,ff422162-fc4a-4b65-a0e2-17f5095ea2c6,2 и более заказов


In [78]:
# Добавим к основной таблице таблицу с покупателями, совершивших 2 и более заказов
data = data.merge(repeat_customers, on='customer_id', how='left')
data.drop('index', axis= 1 , inplace= True)

In [79]:
# Появившиеся NaN соответствуют покупателям с 1 заказом. Добавим эту информацию.
data['customer_type'] = data['customer_type'].fillna('1 заказ')
data

,date,customer_id,order_id,product,quantity,price,month,price_total,customer_type
0,2018-10-01 00:00:00,ee47d746-6d2f-4d3c-9622-c31412542920,68477,"комнатное растение в горшке алое вера, d12, h30",1,142.0,2018-10-01 00:00:00,142.0,1 заказ
1,2018-10-01 00:00:00,ee47d746-6d2f-4d3c-9622-c31412542920,68477,"комнатное растение в горшке кофе арабика, d12, h25",1,194.0,2018-10-01 00:00:00,194.0,1 заказ
2,2018-10-01 00:00:00,ee47d746-6d2f-4d3c-9622-c31412542920,68477,радермахера d-12 см h-20 см,1,112.0,2018-10-01 00:00:00,112.0,1 заказ
3,2018-10-01 00:00:00,ee47d746-6d2f-4d3c-9622-c31412542920,68477,хризолидокарпус лутесценс d-9 см,1,179.0,2018-10-01 00:00:00,179.0,1 заказ
4,2018-10-01 00:00:00,ee47d746-6d2f-4d3c-9622-c31412542920,68477,циперус зумула d-12 см h-25 см,1,112.0,2018-10-01 00:00:00,112.0,1 заказ
...,...,...,...,...,...,...,...,...,...
5567,2020-01-30 21:00:00,63208953-a8e4-4f77-9b47-3a46e7b72eee,104002,томата (помидор) черниченский черри № 116 сорт индетерминантный позднеспелый черный,2,38.0,2020-01-30 21:00:00,76.0,2 и более заказов
5568,2020-01-30 22:00:00,d99d25f1-4017-4fcd-8d29-c580cc695a1a,107336,дендробиум санок анна грин 1 ствол d-12 см,1,869.0,2020-01-30 22:00:00,869.0,2 и более заказов
5569,2020-01-31 02:00:00,2c9bd08d-8c55-4e7a-9bfb-8c56ba42c6d6,106336,подставка для обуви резиновая attribute 80x40 см amc080,1,354.0,2020-01-31 02:00:00,354.0,2 и более заказов
5570,2020-01-31 12:00:00,cdd17932-623e-415f-a577-3b31312fd0e2,102002,тагетис крупноцветковый рассада однолетних цветов в кассете по 6 шт,1,128.0,2020-01-31 12:00:00,128.0,2 и более заказов


In [80]:
# Определим, сколько у нас повторных и одиночных покупателей 
data_type = data.groupby('customer_type')['customer_id'].nunique().sort_values(ascending=False).reset_index()
data_type.columns = ['customer_type', 'customer_id_count']
data_type

,customer_type,customer_id_count
0,1 заказ,1544
1,2 и более заказов,884


In [81]:
fig = px.bar(data_type, x='customer_type', y='customer_id_count')
fig.update_layout(title='Распределение покупателей по количеству заказов',\
                  xaxis_title='Тип покупателя', yaxis_title='Количество')
fig.show()

In [82]:
fig= go.Figure(data=[go.Pie(labels=data_type['customer_type'], values=data_type['customer_id_count'])])
fig.update_layout(title='Процентное распределение покупателей по количеству заказов')
fig.show()

Около 64% покупателей совершили только 1 заказ. Остальных покупателей можно назвать постоянными.

### Динамика показателей по месяцам

Рассмотрим ряд показателей во времени с отображением типа пользователя.

In [83]:
# Сгруппируем таблицу по месяцу и типу покупателя. Добавим новый столбец со средним чеком заказа check_avg
data_by_month_type = data.groupby(['month', 'customer_type']).agg({'price_total':'sum', 'order_id':'nunique'}).reset_index()
data_by_month_type.columns = ['month', 'customer_type' , 'revenue', 'order_id_count']
data_by_month_type['check_avg'] = data_by_month_type['revenue'] / data_by_month_type['order_id_count']
data_by_month_type

,month,customer_type,revenue,order_id_count,check_avg
0,2018-10-01 00:00:00,1 заказ,1037.0,1,1037.0
1,2018-10-01 08:00:00,1 заказ,824.0,1,824.0
2,2018-10-01 08:00:00,2 и более заказов,269.0,1,269.0
3,2018-10-01 09:00:00,2 и более заказов,674.0,1,674.0
4,2018-10-01 11:00:00,1 заказ,820.0,1,820.0
...,...,...,...,...,...
2968,2020-01-30 21:00:00,2 и более заказов,76.0,1,76.0
2969,2020-01-30 22:00:00,2 и более заказов,869.0,1,869.0
2970,2020-01-31 02:00:00,2 и более заказов,354.0,1,354.0
2971,2020-01-31 12:00:00,2 и более заказов,128.0,1,128.0


In [84]:
# Помесячная выручка
fig = px.bar(data_by_month_type, x='month', y='revenue', color='customer_type',\
             labels={'customer_type':'Количество заказов у покупателя'})
fig.update_layout(title='Динамика выручки по месяцам', xaxis_title='Месяц', yaxis_title='Выручка')
fig.show()

Максимальных значений общая выручка достигала в ноябре и декабре 2018 (около 360 тыс и 351 тыс), а также в апреле 2019 (около 315 тыс). В последующие месяцы выручка уменьшилась, в ноябре 2019 наблюдается провал (127 тыс). 

На графике можем заметить крайне малое количество новых покупателей в ноябре2019-январе2020. Это может быть связано с проблемами в рекламе и поисковой выдаче сайта.

In [85]:
# Количество заказов по месяцам
fig = px.bar(data_by_month_type, x='month', y='order_id_count', color='customer_type',\
             labels={'customer_type':'Количество заказов у покупателя'})
fig.update_layout(title='Количество заказов по месяцам', xaxis_title='Месяц', yaxis_title='Количество')
fig.show()

По графику видим, что много заказов пришлось на декабрь 2018 (283 заказа), февраль 2019 (288 заказов), апрель 2019 (267 заказов). Затем, после спада постепенно достигая апрельского значения к январю 2020 г (269 заказов).

Рост количества заказов к концу анализируемого периода при одновременном снижении выручки может говорить об изменении структуры покупательской корзины.

In [86]:
# Средний чек заказа по месяцам
data_by_month = data.groupby('month').agg({'price_total':'sum', 'order_id':'nunique'}).reset_index()
data_by_month.columns = ['month', 'revenue', 'order_id_count']
data_by_month['check_avg'] = data_by_month['revenue'] / data_by_month['order_id_count']

fig = px.bar(data_by_month, x='month', y='check_avg')
fig.update_layout(title='Средний чек заказа по месяцам', xaxis_title='Месяц', yaxis_title='Стоимость')
fig.show()

По графику видим, что в ноябре 2018 была самая высокая стоимость среднего чека (1761). Ближе к концу исследуемого периода уровень среднего чека понизился до 646.

**Вывод**

- Всего наименований продуктов: 2337
- Всего заказов: 3518
- Всего уникальных покупателей: 2428
- В среднем, на одного покупателя приходится 1 заказ, а выручка составляет 838 на покупателя.
- Выделили группу лояльных покупателей, т.е. покупателей с 2 и более заказами. Оказалось что доля таких покупателей составляет около 36% от всего количества покупателей.
- Максимальных значений общая выручка достигала в ноябре и декабре 2018 (около 360 тыс и 351 тыс), а также в апреле 2019 (около 315 тыс). В последующие месяцы выручка уменьшилась, в ноябре 2019 наблюдается провал (127 тыс). Ниже проверим гипотезу о равенстве помесячной выручки от повторных и одиночных покупателей.

- Выяснили, что в ноябре2019-январе2020 было крайне мало новых покупателей. Это может быть связано с проблемами в рекламе и поисковой выдаче сайта.

- По графику видим, что много заказов пришлось на декабрь 2018 (283 заказа), февраль 2019 (288 заказов), апрель 2019 (267 заказов). Затем, после спада постепенно достигая апрельского значения к январю 2020 г (269 заказов).

- Рост количества заказов к концу анализируемого периода при одновременном снижении выручки может говорить об изменении структуры покупательской корзины.

- В ноябре 2018 была самая высокая стоимость среднего чека (1761). Ближе к концу исследуемого периода уровень среднего чека понизился до 650.

## Сегментация пользователей
### Разбивка товаров на категории

Чтобы разделить товары на категории, рассмотрим сначала самые популярные товары и начнем им присваивать категории.

In [87]:
data_top_products = data['product'].value_counts().reset_index()
data_top_products.columns = ['product_name', 'product_quantity']
data_top_products = data_top_products.loc[data_top_products['product_quantity'] > 10]
data_top_products

,product_name,product_quantity
0,пеларгония розебудная red pandora укорененный черенок,66
1,пеларгония розебудная prins nikolai укорененный черенок,56
2,пеларгония зональная диам. 12 см сиреневый полумахровый,53
3,сумка-тележка 2-х колесная gimi argo синяя,47
4,пеларгония розебудная mary укорененный черенок,40
5,пеларгония розебудная margaretha укорененный черенок,31
6,пеларгония розебудная queen ingrid укорененный черенок,31
7,пеларгония зональная ринго вайт d-7 см h-10 см укорененный черенок,25
8,пеларгония зональная махровая лососевая,25
9,пеларгония зональная диам. 12 см коралловая полумахровая,24


Видим, что самые популярные товары в магазине - цветы, рассада, тележки. Эти категории точно будем выделять.

Ниже я создала функцию, в которую постепенно добавляла новые слова для разделения товаров на категории. Пользовалась кодом после функции, чтобы вывести самые популярные наименования товаров, которым категория еще не была присвоена. 

In [88]:
def category(row):
    '''
    функция для разделения товаров на категории
    '''
    try:
        if 'кассет' in row or 'черен' in row or 'объем' in row or 'пеларгония' in row or 'бакопа' in row or 'петуния' in row\
        or 'калибрахоа' in row or 'фуксия' in row or 'настурция' in row or 'овсянница' in row or 'котовник' in row\
        or 'вербейник' in row or 'антуриум' in row or 'кашпо' in row or 'примула' in row\
        or 'гвоздика' in row or 'лобелия' in row or 'флокс' in row or 'гипсофила' in row or 'цикламен' in row\
        or 'бегония' in row or 'гортензия' in row or 'калла' in row or 'кипарисовик' in row or 'папоротник' in row\
        or 'ель' in row or 'осина' in row or 'роза' in row or 'пуансеттия' in row or 'незабудка' in row\
        or 'персиколистный' in row or 'вербена' in row or 'львиный зев' in row or 'ранункулус' in row or 'камнеломка' in row\
        or 'виола' in row or 'кореопсис' in row or 'ромашка' in row or 'ясколка' in row or 'космея' in row or 'клен' in row\
        or 'колокольчик' in row or 'лапчатка' in row or 'седум' in row or 'тюльпан' in row or 'физостегия' in row\
        or 'хризантема' in row or 'календула' in row or 'пиретрум' in row or 'бархатцы' in row\
        or 'цинния' in row or 'фиалк' in row or 'эшшольция' in row or 'цинерария' in row or 'годеция' in row\
        or 'дендробиум' in row or 'георгин' in row or 'смолевка' in row or 'лаватера' in row or 'ципера' in row\
        or 'декабрист' in row or 'хлорофитум' in row or 'эвкалипт' in row or 'афеляндра' in row or 'циперус' in row\
        or 'алоэ' in row or 'цвет' in row or 'мединилла' in row or 'мимоза' in row or 'фаленопсис' in row\
        or 'алое' in row or 'лаванда' in row or 'мирт' in row or 'каланхое' in row or 'нефролепис' in row\
        or 'калатея' in row or 'кофе' in row or 'кактус' in row or 'аптения' in row or 'амариллис' in row\
        or 'эхеверия' in row or 'гардения' in row or 'чабер' in row or 'шеффлера' in row or 'синнингия' in row\
        or 'пуансетия' in row or 'спатифиллум' in row or 'муррайя' in row or 'хамедорея' in row or 'азалия' in row\
        or 'молодило' in row or 'лантана' in row or 'раст' in row or 'монарда' in row or 'гербера' in row or 'соланум' in row\
        or 'хризолидокарпус' in row or 'гиацинт' in row or 'цитрофортунелла' in row or 'аспарагус' in row or 'крокус' in row\
        or 'полынь' in row or 'фиттония' in row or 'пахира' in row or 'импатиенс' in row or 'суккулент' in row\
        or 'литопс' in row or 'альбука' in row or 'фикус' in row or 'джункус' in row or 'перемория' in row\
        or 'замиокулькас' in row or 'глоксиния' in row or 'пеперомия' in row or 'драцена' in row or 'юкка' in row\
        or 'эпипремнум' in row or 'буддлея' in row or 'крассула' in row or 'фатсия' in row or 'иссоп' in row\
        or 'мускари' in row or 'капсикум' in row or 'адиантум' in row or 'гимнокалициум' in row or 'радермахера' in row\
        or 'сантолина' in row or 'скиммия' in row:    
            return 'ЦВЕТЫ'
        elif 'томат' in row or 'арбуз' in row or 'дын' in row or 'укроп' in row or 'клубника' in row or 'базилик' in row\
        or 'тимьян' in row or 'петрушка' in row or 'огурец' in row or 'сельдерей' in row or 'лавр' in row  \
        or 'виноград' in row or 'салат' in row or 'капуст' in row or 'морков' in row or 'горох' in row or 'земляник' in row\
        or 'прян' in row or 'розмарин' in row or 'тыкв' in row or 'клубник' in row or 'перц' in row or 'кабачок' in row\
        or 'душица' in row or 'бальзамин' in row or 'зверобой' in row or 'мелисса' in row or 'мята' in row\
        or 'патиссон' in row or 'подсолнечник' in row:
            return 'ОВОЩНЫЕ КУЛЬТУРЫ И ПРЯНОСТИ'
        elif 'теле' in row or 'багаж' in row or 'сумк' in row:
            return 'СУМКИ И ТЕЛЕЖКИ'
        elif 'муляж' in row or 'искус' in row:
            return 'МУЛЯЖИ'
        elif 'сиденье' in row or 'чехол' in row or 'карниз' in row or 'ванн' in row or 'плечики' in row \
        or 'подрукавник' in row or 'окон' in row or 'коврик' in row or 'ковер' in row or 'корзина' in row or 'подставка' in row\
        or 'полк' in row or 'стремянк' in row or 'этажерк' in row or 'ящик' in row or 'вешал' in row\
        or 'термометр' in row or 'комод' in row or 'коробк' in row or 'бак' in row or 'весы напольные' in row\
        or 'глаж' in row or 'ключница' in row or 'пакет' in row or 'весы настольные' in row or 'обувница' in row\
        or 'пуф' in row or 'светильник' in row or 'фен' in row or 'утюг' in row or 'корзинк' in row or 'фоторамка' in row\
        or 'сушилк' in row or 'унитаз' in row or 'таз' in row or 'корыто' in row or 'гладил' in row\
        or 'скатерть' in row or 'салфетка' in row or 'простын' in row or 'подушк' in row or 'наматрасник' in row\
        or 'покрывало' in row or 'халат' in row or 'махров' in row or 'одеяло' in row or 'наволочк' in row\
        or 'плед' in row or 'пододеяльник' in row or 'матрац' in row or 'одежд' in row:
            return 'ДЛЯ ДОМА'
        elif 'кувшин' in row or 'банка' in row or 'круж' in row or 'салатник' in row or 'овощеварка' in row\
        or 'просеиватель' in row or 'контейнер' in row or 'вилка' in row or 'миска' in row or 'сковорода' in row\
        or 'измельчитель' in row or 'хлебница' in row or 'тарелка' in row or 'сахарница' in row or 'блюдо' in row\
        or 'ложка' in row or 'разделочн' in row or 'чайн' in row or 'нож' in row or 'кух' in row or 'кастрюл' in row\
        or 'кекс' in row or 'терка' in row or 'блюдце' in row or 'сотейник' in row  or 'фужер' in row or 'сковород' in row\
        or 'сито' in row or 'орехоколка' in row or 'миксер' in row or 'отделитель' in row or 'подголовник' in row\
        or 'кофр' in row or 'толкушка' in row or 'стеллаж' in row or 'стакан' in row or 'ковш' in row\
        or 'соковыжималка' in row or 'масленка' in row or 'картофелемялка' in row or 'бидон' in row or 'скалк' in row\
        or 'бокал' in row or 'бульонница' in row or 'соковарка' in row or 'емкость' in row or 'посуд' in row\
        or 'половник' in row or 'лопатк' in row or 'противень' in row or 'столов' in row or 'кисточка' in row\
        or 'мантоварка' in row or 'выпечк' in row or 'тортница' in row or 'венчик' in row or 'кондитер' in row\
        or 'пресс' in row or 'лоток' in row or 'электроштопор' in row or 'кипятильник' in row or 'пароварк' in row:
            return 'ДЛЯ КУХНИ'
        elif 'щетка' in row or 'швабр' in row or 'тряпк' in row or 'чистк' in row or 'вантуз' in row\
        or 'ведр' in row or 'ерш' in row or 'перчатк' in row or 'мыл' in row or 'веник' in row or 'салфет' in row\
        or 'совок' in row or 'мытья' in row or 'стирк' in row or 'средство' in row or 'чист' in row\
        or 'скребок' in row or 'антижир' in row or 'пылесос' in row or 'губка' in row or 'ткань' in row or 'совк' in row: 
            return 'ДЛЯ УБОРКИ'
        elif 'бензин' in row or 'линейка' in row or  'штангенциркуль' in row\
        or 'сварка' in row or 'термос' in row or 'крепеж' in row or 'крючок' in row or 'кольца' in row\
        or 'прищеп' in row or 'ручка' in row or 'петля' in row or 'зажигалк' in row or 'сверло' in row\
        or 'урна' in row or 'инструмент' in row or 'сверел' in row or 'пробк' in row or 'напильник' in row\
        or 'шило' in row or 'насадк' in row or 'решетк' in row or 'котел' in row or 'блок' in row or 'бобин' in row\
        or 'фиксатор' in row or 'шнур' in row or 'шпингалет' in row or 'завертка' in row or 'шпагат' in row or 'паста' in row\
        or 'фал' in row or 'маска' in row:
            return 'ПРОЧЕЕ'
    except:
        return 'нет категории'
    
data['category'] = data['product'].apply(category)

In [89]:
# Код ниже использовался для ручного присвоения категории товарам
#data_no_category = data[(data['category'] != 'ЦВЕТЫ') & (data['category'] != 'ОВОЩНЫЕ КУЛЬТУРЫ И ПРЯНОСТИ')\
#               & (data['category'] != 'СУМКИ И ТЕЛЕЖКИ') & (data['category'] != 'МУЛЯЖИ') & (data['category'] != 'ДЛЯ ДОМА')\
#               & (data['category'] != 'ДЛЯ КУХНИ') & (data['category'] != 'ДЛЯ УБОРКИ') & (data['category'] != 'ПРОЧЕЕ')]
#data_top_products = data_no_category['product'].value_counts().reset_index()
#data_top_products.columns = ['product_name', 'product_quantity']
#data_top_products = data_top_products.loc[data_top_products['product_quantity'] > 1]
#data_top_products
#data_no_category

In [90]:
# Получилось 8 категорий
data['category'].unique()

array(['ЦВЕТЫ', 'ДЛЯ ДОМА', 'ПРОЧЕЕ', 'ДЛЯ КУХНИ', 'СУМКИ И ТЕЛЕЖКИ',
       'МУЛЯЖИ', 'ОВОЩНЫЕ КУЛЬТУРЫ И ПРЯНОСТИ', 'ДЛЯ УБОРКИ'],
      dtype=object)

`ЦВЕТЫ` - Выделена отдельная категория, включающая как черенки цветов для высадки в грунт, так и комнатные цветы. Не стала их разделять, так как какие-то цветы можно держать и дома, а летом выносить в горшках на улицу. То есть их нельзя четко разделить на те, которые покупают в определенное время года.

`ОВОЩНЫЕ КУЛЬТУРЫ И ПРЯНОСТИ` - сезонная категория, включающая в себя рассаду или семена овощей, ягоды, тыквинных, трав. Товары данной категории относятся к низкому ценовому сегменту и обычно покупаются перед началом дачного сезона.

`МУЛЯЖИ` - выделено в отдельную категорию, включающую искусственные цветы и овощи/фрукты.

`СУМКИ И ТЕЛЕЖКИ` - малочисленная группа, хотя несколько позиций входит в топовые по количеству заказов.

`ДЛЯ ДОМА` - разнообразные товары для дома, такие как мебель, товары для хранения, декора, а также текстиль.

`ДЛЯ КУХНИ`, `ДЛЯ УБОРКИ` - по сути это тоже товары для дома, но из-за большого количества позиций решено было выделить в отдельные категории.

### Характеристики категорий товаров

In [91]:
# Количество товаров в категориях и выручка по категориям
data_by_category = data.groupby('category').agg({'product':'nunique', 'price_total':'sum'}).reset_index()
data_by_category.columns= ['category', 'product_count', 'revenue']
data_by_category['avg_price'] = data_by_category['revenue'] / data_by_category['product_count']
data_by_category = data_by_category.sort_values(by='product_count', ascending = False)
data_by_category

,category,product_count,revenue,avg_price
7,ЦВЕТЫ,999,1.104941e+06,1106.047047
0,ДЛЯ ДОМА,537,1.306486e+06,2432.934823
1,ДЛЯ КУХНИ,262,3.250480e+05,1240.641221
4,ОВОЩНЫЕ КУЛЬТУРЫ И ПРЯНОСТИ,243,8.194700e+04,337.230453
6,СУМКИ И ТЕЛЕЖКИ,112,8.417152e+05,7515.314732
2,ДЛЯ УБОРКИ,73,1.112105e+05,1523.431507
5,ПРОЧЕЕ,57,9.733700e+04,1707.666667
3,МУЛЯЖИ,54,7.702667e+04,1426.419753


In [92]:
# Сколько товаров в каждой категории
fig = px.bar(data_by_category, x='category', y='product_count')
fig.update_layout(title='Распределение товаров по категориям', xaxis_title='Категория товара', yaxis_title='Количество')
fig.show()

In [93]:
fig= go.Figure(data=[go.Pie(labels=data_by_category['category'], values=data_by_category['product_count'])])
fig.update_layout(title='Процентное распределение товаров по категориям')
fig.show()

По графикам выше видим, что самая многочисленная категория - ЦВЕТЫ (около 43%). Вместе с ОВОЩНЫМИ КУЛЬТУРАМИ И ПРЯНОСТЯМИ эти товары составляют около 53% от общего ассортимента товаров. Остальные категории - различные товары быта. Можно выделить категорию товаров ДЛЯ ДОМА (23%) и ДЛЯ КУХНИ (11%). Категория СУМКИ И ТЕЛЕЖКИ составляют чуть меньше 5%.

In [94]:
# Выручка по категориям
fig = px.bar(data_by_category, x='category', y='revenue')
fig.update_layout(title='Выручка по категориям за период', xaxis_title='Категория товара', yaxis_title='Выручка',\
                  xaxis ={'categoryorder':'total descending'})
fig.show()

Видим, что больше всего выручка у товаров ДЛЯ ДОМА, затем у товаров ЦВЕТЫ, СУМКИ И ТЕЛЕЖКИ.

In [95]:
# Средняя выручка на SKU по категориям за период
fig = px.bar(data_by_category, x='category', y='avg_price')
fig.update_layout(title='Средняя выручка на SKU по категориям за период', xaxis_title='Категория товара',\
                  yaxis_title='Средняя выручка', xaxis ={'categoryorder':'total descending'})
fig.show()

В категории `СУМКИ И ТЕЛЕЖКИ` небольшой ассортимент представленных товаров. Зато несколько позиций входит в топовые по количеству заказов. Выше всего средняя выручка на SKU - в категории СУМКИ И ТЕЛЕЖКИ.

### Анализ продуктовой корзины для повторных и одиночных пользователей

Рассмотрим отдельно покупателей с 2 и более заказами. Выясним, сколько помесячно единиц товаров по категориям покупают повторные покупатели.

In [96]:
# Покупатели с 2 и более заказами. 
data_repeat = data[data['customer_type'] == '2 и более заказов']

In [97]:
data_by_month_cat_repeat = data_repeat.groupby(['month', 'category'])\
                    .agg({'price_total':'sum', 'quantity':'sum'}).reset_index()
data_by_month_cat_repeat.columns = ['month', 'category', 'revenue', 'quantity_count']

# Количество заказов по месяцам с разбивкой на категории
fig = px.bar(data_by_month_cat_repeat.sort_values(by='category'), x='month', y='quantity_count', color='category',\
             labels={'category':'Категория'})
fig.update_layout(title='Количество купленных единиц товаров по месяцам', xaxis_title='Месяц', yaxis_title='Количество')
fig.show()

In [98]:
data_by_month_cat_repeat

,month,category,revenue,quantity_count
0,2018-10-01 08:00:00,ДЛЯ ДОМА,269.0,1
1,2018-10-01 09:00:00,ДЛЯ ДОМА,674.0,1
2,2018-10-01 12:00:00,ЦВЕТЫ,614.0,1
3,2018-10-01 13:00:00,ДЛЯ ДОМА,187.0,1
4,2018-10-01 16:00:00,ДЛЯ ДОМА,188.0,1
...,...,...,...,...
1840,2020-01-30 21:00:00,ОВОЩНЫЕ КУЛЬТУРЫ И ПРЯНОСТИ,76.0,2
1841,2020-01-30 22:00:00,ЦВЕТЫ,869.0,1
1842,2020-01-31 02:00:00,ДЛЯ ДОМА,354.0,1
1843,2020-01-31 12:00:00,ЦВЕТЫ,128.0,1


Мы видим, что основные покупки повторных покупателей приходятся на категории ЦВЕТЫ и ОВОЩНЫЕ КУЛЬТУРЫ И ПРЯНОСТИ. Большой пик приходится на апрель (начало дачного сезона). Одновременно с этим докупают что-то для дома. Но затем происходит затишье во время дачного сезона (июнь-октябрь). А далее в осенне-зимний период дачники опять возвращаются и обращают внимание на категории ЦВЕТЫ и товары ДЛЯ ДОМА.
Также видим, что последние 3 месяца возрос интерес к товарам ДЛЯ КУХНИ.

А теперь посмотрим одиночных покупателей. То есть тех, которые после одного заказа не возвращались.

In [99]:
# Покупатели с 1 заказом. 
data_single = data[data['customer_type'] == '1 заказ']

In [100]:
data_by_month_cat_single = data_single.groupby(['month', 'category'])\
                    .agg({'price_total':'sum', 'quantity':'sum'}).reset_index()
data_by_month_cat_single.columns = ['month', 'category', 'revenue', 'quantity_count']

# Количество заказов по месяцам с разбивкой на категории
fig = px.bar(data_by_month_cat_single.sort_values(by='category'), x='month', y='quantity_count', color='category',\
             labels={'category':'Категория'})
fig.update_layout(title='Количество купленных единиц товаров по месяцам', xaxis_title='Месяц', yaxis_title='Количество')
fig.show()

In [101]:
data_by_month_cat_single 

,month,category,revenue,quantity_count
0,2018-10-01 00:00:00,ЦВЕТЫ,1037.0,7
1,2018-10-01 08:00:00,ЦВЕТЫ,824.0,1
2,2018-10-01 11:00:00,ЦВЕТЫ,820.0,10
3,2018-10-01 15:00:00,ПРОЧЕЕ,3648.0,96
4,2018-10-01 18:00:00,ДЛЯ КУХНИ,1325.0,3
...,...,...,...,...
1644,2020-01-03 09:00:00,ЦВЕТЫ,97.0,1
1645,2020-01-04 16:00:00,ДЛЯ ДОМА,1799.0,1
1646,2020-01-05 21:00:00,ДЛЯ КУХНИ,179.0,1
1647,2020-01-08 17:00:00,ЦВЕТЫ,60.0,1


Мы видим, что одиночные покупатели тоже приобретают ЦВЕТЫ. А также перед началом сезона ОВОЩНЫЕ КУЛЬТУРЫ И ПРЯНОСТИ (апрель и май).
Также видим, что в ноябре покупатели приходили за муляжами. Могу предположить, что искусственные овощи/фрукты разово закупали в качестве учебного пособия для художественных школ/институтов. 

В ноябре2019-январе2020 одиночных покупателей не было. Как выше отмечала, это может быть связано с проблемами в рекламе и поисковой выдаче сайта. На это необходмио обратить внимание менеджера.

### Распределение количества заказов среди повторных и одиночных покупателей по категориям товаров

In [102]:
# Посмотрим, какую долю занимают повторные покупатели по количеству заказов товаров, распределенных по категориям
data_cat_type_share = data.pivot_table(index='category', columns='customer_type', values='order_id', aggfunc='nunique')\
                                                             .reset_index()
data_cat_type_share['total'] = data_cat_type_share['1 заказ'] + data_cat_type_share['2 и более заказов']
data_cat_type_share['share'] = round(data_cat_type_share['2 и более заказов'] / data_cat_type_share['total'], 2)
data_cat_type_share = data_cat_type_share.sort_values(by='share', ascending=False)
data_cat_type_share

customer_type,category,1 заказ,2 и более заказов,total,share
6,СУМКИ И ТЕЛЕЖКИ,163,222,385,0.58
0,ДЛЯ ДОМА,383,514,897,0.57
2,ДЛЯ УБОРКИ,47,56,103,0.54
7,ЦВЕТЫ,704,832,1536,0.54
1,ДЛЯ КУХНИ,174,186,360,0.52
4,ОВОЩНЫЕ КУЛЬТУРЫ И ПРЯНОСТИ,114,117,231,0.51
5,ПРОЧЕЕ,36,38,74,0.51
3,МУЛЯЖИ,69,37,106,0.35


In [103]:
fig=px.bar(data_cat_type_share, x='category', y='total', color='share',\
           labels={'share':'Доля покупателей с 2 заказами и более'})
fig.update_layout(title='Доля повторных покупателей по категориям', xaxis_title='Категория товара',\
                  yaxis_title='Общее количество заказов', xaxis ={'categoryorder':'total descending'})
fig.show()

Мы видим, что муляжи покупают, в основном, одиночные покупатели (например, разово для учебы). А вот покупатели товаров для дома и сумок с тележками - в основном повторные покупатели.

**Вывод**

- Выделили 8 категорий товаров

- `ЦВЕТЫ` - самая многочисленная группа (981 позиция товаров).

`ЦВЕТЫ` + `ОВОЩНЫЕ КУЛЬТУРЫ И ПРЯНОСТИ` - около 53% от всего ассортимента.

`ДЛЯ ДОМА` + `ДЛЯ КУХНИ` + `ДЛЯ УБОРКИ` + `МУЛЯЖИ` - около 42%.

`СУМКИ И ТЕЛЕЖКИ` - около 5%.

- Больше всего выручки принесли товары ДЛЯ ДОМА (окло 1158000), затем товары ЦВЕТЫ (1049000), СУМКИ И ТЕЛЕЖКИ (779000).

- Выше всего средняя выручка на SKU - в категории СУМКИ И ТЕЛЕЖКИ.

- Основные покупки повторных покупателей приходятся на категории ЦВЕТЫ и ОВОЩНЫЕ КУЛЬТУРЫ И ПРЯНОСТИ. Перед началом дачного сезона (апрель-май) количество заказов из этих категорий резко возрастает. Одновременно с этим докупают что-то для дома. Далее следует снижение покупательской активности во время дачного сезона (июнь-октябрь). Затем в осенне-зимний период дачники опять возвращаются и обращают внимание на категории ЦВЕТЫ и товары ДЛЯ ДОМА. Также видим, что последние 3 месяца возрос интерес к товарам ДЛЯ КУХНИ.

- Одиночные покупатели тоже приобретают ЦВЕТЫ. А также перед началом сезона ОВОЩНЫЕ КУЛЬТУРЫ И ПРЯНОСТИ (апрель и май).
Также видим, что в ноябре покупатели приходили за муляжами. Могу предположить, что искусственные овощи/фрукты разово закупали в качестве учебного пособия для художественных школ/институтов. 

## Статистические гипотезы
### Статистическая значимость различий в помесячной выручке от повторных и одиночных покупателей.

**Нулевая гипотеза:**
- H0 - средняя помесячная выручка от повторных и одиночных покупателей одинакова.

**Альтернативная двухсторонняя гипотеза:**
- Н1 - средняя помесячная выручка от повторных и одиночных покупателей различается.

In [104]:
data_by_month_type.head(6)

,month,customer_type,revenue,order_id_count,check_avg
0,2018-10-01 00:00:00,1 заказ,1037.0,1,1037.0
1,2018-10-01 08:00:00,1 заказ,824.0,1,824.0
2,2018-10-01 08:00:00,2 и более заказов,269.0,1,269.0
3,2018-10-01 09:00:00,2 и более заказов,674.0,1,674.0
4,2018-10-01 11:00:00,1 заказ,820.0,1,820.0
5,2018-10-01 12:00:00,2 и более заказов,614.0,1,614.0


In [105]:
revenue_single = data_by_month_type[data_by_month_type['customer_type'] == '1 заказ']['revenue']
revenue_repeat = data_by_month_type[data_by_month_type['customer_type'] == '2 и более заказов']['revenue']

In [106]:
alpha = 0.05
results = st.ttest_ind(revenue_repeat, revenue_single)
print ('P-value:', results.pvalue)
if results.pvalue < alpha:
    print('Отвергаем Н0')
else:
    print('Не отвергаем Н0')

P-value: 0.0032631568222471277
Отвергаем Н0


In [107]:
print(revenue_repeat.mean())
revenue_single.mean()

1208.3002001685632


1462.3871315600288

**Вывод**: не отвергаем гипотезу о том, что средняя помесячная выручка от повторных и одиночных покупателей одинакова, так как полученное значение p-value превышает заданный уровень значимости. Cредняя помесячная выручка двух групп покупателей статистически значимо не отличается друг от друга.

### Статистическая значимость различий в средней выручке на товар между категориями.

Проверим гипотезу для смежных категорий ЦВЕТЫ и ОВОЩНЫЕ КУЛЬТУРЫ И ПРЯНОСТИ.

**Нулевая гипотеза:**
- H0 - средняя выручка на товар в категории ЦВЕТЫ и категории ОВОЩНЫЕ КУЛЬТУРЫ И ПРЯНОСТИ одинаковая.

**Альтернативная двухсторонняя гипотеза:**
- H1 - средняя выручка на товар в категории ЦВЕТЫ и категории ОВОЩНЫЕ КУЛЬТУРЫ И ПРЯНОСТИ разная.

In [108]:
data_by_product = data.pivot_table(index=['product','category'], values='price_total', aggfunc='sum').reset_index()
data_by_product

,product,category,price_total
0,tepмокружка avex freeflow 700 мл зеленый avex0759,ДЛЯ КУХНИ,4798.0
1,tepмокружка avex freeflow 700 мл сталь avex0776,ДЛЯ КУХНИ,2399.0
2,tepмокружка avex recharge 500 мл голубой avex0681,ДЛЯ КУХНИ,4198.0
3,автоматическая щетка leifheit для мытья окон с ручкой 43 см. 51114,ОВОЩНЫЕ КУЛЬТУРЫ И ПРЯНОСТИ,14458.0
4,агератум рассада однолетних цветов в кассете по 10 шт,ЦВЕТЫ,420.0
...,...,...,...
2332,"ясколка войлочная белая объем 0,5 л",ЦВЕТЫ,291.0
2333,"ящик для хранения textilebox curver 5,7 л 34x20x13 см прозрачный 03003-001-00",ДЛЯ ДОМА,896.0
2334,ящик для хранения фимако люкс плетеный с крышкой m бежевый,ДЛЯ ДОМА,1498.0
2335,ящик почтовый металлический с врезным замком почта 1205250,ДЛЯ ДОМА,277.0


In [109]:
data_by_cat_flowers = data_by_product[data_by_product['category'] == 'ЦВЕТЫ']['price_total']
data_by_cat_veget= data_by_product[data_by_product['category'] == 'ОВОЩНЫЕ КУЛЬТУРЫ И ПРЯНОСТИ']['price_total']

In [110]:
alpha = 0.05
results = st.ttest_ind(data_by_cat_flowers, data_by_cat_veget)
print ('P-value:', results.pvalue)
if results.pvalue < alpha:
    print('Отвергаем Н0')
else:
    print('Не отвергаем Н0')

P-value: 8.25690228705241e-07
Отвергаем Н0


**Вывод**: отвергаем гипотезу о том, что средняя выручка на товар в категории ЦВЕТЫ и в категории ОВОЩНЫЕ КУЛЬТУРЫ И ПРЯНОСТИ одинаковая, так как полученное значение p-value ниже заданного уровня значимости.

Есть все основания говорить, что средняя выручка на товар в категориях различна.

## Подготовка выводов и рекомендаций

**Предварительно была выполнена предобработка данных:**

- Определены и устранены дубликаты, тестовый заказ.
- Скорректирован тип данных в столбце с датой.
- Добавлены новые столбцы `month` и `price_total`.
- Устранены некоторые выбросы в данных.

**Проведен детализированный анализ данных:**

- Всего наименований продуктов: 2337, заказов: 3518, уникальных покупателей: 2428
- В среднем, на одного покупателя приходится 1 заказ, а выручка составляет 838 на покупателя.

- Максимальных значений общая выручка достигала в ноябре и декабре 2018 (около 360 тыс и 351 тыс), а также в апреле 2019 (около 315 тыс). В последующие месяцы выручка уменьшилась, в ноябре 2019 наблюдается провал (127 тыс).
- Выяснили, что в ноябре2019-январе2020 было крайне мало новых покупателей. Это может быть связано с проблемами в рекламе и поисковой выдаче сайта.
- По графику видим, что много заказов пришлось на декабрь 2018 (283 заказов), февраль 2019 (288 заказов), апрель 2019 (267 заказов). Затем, после спада постепенно достигая апрельского значения к январю 2020 г (269 заказов).
- Наблюдается рост количества заказов к концу анализируемого периода при одновременном снижении выручки, что может говорить об изменении структуры покупательской корзины.

- В ноябре 2018 была самая высокая стоимость среднего чека (1761). Ближе к концу исследуемого периода уровень среднего чека понизился до 650.

**Проведена сегментация товаров по категориям:**

- Выделили 8 категорий товаров

- `ЦВЕТЫ` - самая многочисленная группа (981 позиция товаров).

`ЦВЕТЫ` + `ОВОЩНЫЕ КУЛЬТУРЫ И ПРЯНОСТИ` - около 53% от всего ассортимента.

`ДЛЯ ДОМА` + `ДЛЯ КУХНИ` + `ДЛЯ УБОРКИ` + `МУЛЯЖИ` - около 42%.

`СУМКИ И ТЕЛЕЖКИ` - около 5%.

- Больше всего выручки принесли товары ДЛЯ ДОМА (около 1306486), затем товары ЦВЕТЫ (1103570), СУМКИ И ТЕЛЕЖКИ (841715).

- Выше всего средняя выручка на SKU - в категории СУМКИ И ТЕЛЕЖКИ.

**Проведена сегментация покупателей по количеству заказов:**
- Выделили группу лояльных покупателей, т.е. покупателей с 2 и более заказами. Это повторные покупатели. Покупатели с 1 заказом - одиночные покупатели. Доля повторных покупателей составляет около 36% от общего количества.

- Основные покупки повторных покупателей приходятся на категории ЦВЕТЫ и ОВОЩНЫЕ КУЛЬТУРЫ И ПРЯНОСТИ. Перед началом дачного сезона (апрель-май) количество заказов из этих категорий резко возрастает. Одновременно с этим докупают что-то для дома. Далее следует снижение покупательской активности во время дачного сезона (июнь-октябрь).
Затем в осенне-зимний период дачники опять возвращаются и обращают внимание на категории ЦВЕТЫ и товары ДЛЯ ДОМА. Также видим, что последние 3 месяца возрос интерес к товарам ДЛЯ КУХНИ.

- Одиночные покупатели тоже приобретают ЦВЕТЫ. А также перед началом сезона ОВОЩНЫЕ КУЛЬТУРЫ И ПРЯНОСТИ (апрель и май). Также видим, что в ноябре покупатели приходили за муляжами. Могу предположить, что искусственные овощи/фрукты разово закупали в качестве учебного пособия для художественных школ/институтов. 

**Проверены гипотезы:**
- Не можем утверждать, что средняя помесячная выручка от повторных и одиночных покупателей разная. Средняя помесячная выручка от повторных покупателей статистически значимо не отличается от средней помесячной выручки от одиночных покупателей.
- Cредняя выручка на товар в категории ЦВЕТЫ и в категории ОВОЩНЫЕ КУЛЬТУРЫ И ПРЯНОСТИ разная.

**Рекомендации по персонализированным предложениям для групп покупателей:**
- Следует ориентироваться на покупателей категорий ЦВЕТЫ И ОВОЩНЫЕ КУЛЬТУРЫ И ПРЯНОСТИ. Это товары, за которыми возвращаются, особенно перед дачным сезоном. Задача для группы одиночных покупателей - привлечь их для нового заказа. Это могут быть скидки на рассаду и цветы. Повторные покупатели - лояльные покупатели, следует поддерживать их активность в магазине. Помимо скидок на рассаду и цветы, можно обращать их внимание на другие категории товаров.

- Обратить внимание на категорию СУМКИ И ТЕЛЕЖКИ, приносящих самую большую выручку. Возможно, расширить ассортимент. В этой категории высокие цены на товар, поэтому покупка вряд ли будет спонтанной. Следует усилить маркетинговую активность по продвижению сумок и тележек в магазине, чтобы за этой покупкой вернулись. Особенно для покупателей рассады и цветов, которые сумки или тележки еще не приобретали.

- Активность дачников в магазине затихает на летний сезон, а затем возрастает осенью. Можно в конце дачного сезона делать рассылку с интересными предложениями и рекомендациями по товарам из других категорий.